## Product and activity names

So the point is how to separate between the name of a product (e.g. _electricity, low voltage_) and the name of the activity producing it (e.g. _electricity production_).

I don't think there is a good way to do this but let's see.

In [1]:
import brightway2 as bw

In [2]:
bw.projects

Brightway2 projects manager with 14 objects:
	B4B18
	C3BO
	ConseqUncertainty
	HH
	bw2_import_ecoinvent_2017
	bw2_import_ecoinvent_3.3
	bw2_import_ecoinvent_3.4
	bw2_seminar_ex1
	default
	exiobase-SimaPro-import
	exiobase_test
	import from csv file
	testei34
	testtime
Use `projects.report()` to get a report on all projects.

In [3]:
bw.projects.set_current("testei34") # get one where there is ecoinvent
if bw.Database("testdb"):
    bw.Database("testdb").delete()
t_db = bw.Database("testdb")

Here I create a database where I try various combinations of product and activity names

In [4]:
t_db.write({
    ("testdb", "12alksdnaslkj"):{   # Here I use a random sequence of nrs and letter as activity code (like in ecoinvent)
        'name':'Electricity, low voltage',
        'unit': 'kWh', 
        'exchanges': [{
                'input': ('testdb', 'Fuel production'),
                'amount': 2,
                'unit': 'kg',
                'type': 'technosphere'
            },{
                'input': ('testdb', 'Carbon dioxide'),
                'amount': 1,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb', 'Sulphur dioxide'),
                'amount': 0.1,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb', '12alksdnaslkj'), #important to write the same process name in output
                'amount': 10,
                'unit': 'kWh',
                'type': 'production'
            }]
        },
    ('testdb', 'Fuel production'):{ # Here I use a name as activity name
        'name': 'Refined fuel',
        'unit': 'kg',
        'exchanges':[{
                'input': ('testdb', 'Carbon dioxide'),
                'amount': 10,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb', 'Sulphur dioxide'),
                'amount': 2,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb', 'Crude oil'),
                'amount': -50,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb', 'Fuel production'),
                'amount': 1,
                'unit': 'kg',
                'type': 'production'
            }]
    },
    ('testdb', 'Carbon dioxide'):{'name': 'Carbon dioxide', 'unit':'kg', 'type': 'biosphere'},
    ('testdb', 'Sulphur dioxide'):{'name': 'Sulphur dioxide', 'unit':'kg', 'type': 'biosphere'},
    ('testdb', 'Crude oil'):{'name': 'Crude oil', 'unit':'kg', 'type': 'biosphere'}

    })

functional_unit = {t_db.get("12alksdnaslkj") : 1}
lca = bw.LCA(functional_unit) 
lca.lci()

Writing activities to SQLite3 database:
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/12/2018 08:57:43
  Finished: 03/12/2018 08:57:43
  Total time elapsed: 00:00:00
  CPU %: 112.50
  Memory %: 0.72


Let's get an activity from ecoinvent and from this database so we can compare

In [5]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.4 conseq
	testdb

In [6]:
mydb = bw.Database('ecoinvent 3.4 conseq')
myact = mydb.random()
print(myact)

'heat and power co-generation unit construction, 500kW electrical, components for electricity only' (unit, RoW, None)


In [7]:
myact2 = t_db.get("12alksdnaslkj")

Let's compare the two

In [10]:
for exc in list(myact.exchanges()):
    print(exc['type'])
    print(exc['input'])
    print(exc.input) # means 'from'
    print(exc.output) # means 'to'
    print('***')

technosphere
('ecoinvent 3.4 conseq', 'c0972aee6c3244a5b7a0c6690e56f8f8')
'market for assembly of generator and motor, auxilliaries and energy use, heat and power co-generation unit, 160kW' (unit, GLO, None)
'heat and power co-generation unit construction, 500kW electrical, components for electricity only' (unit, RoW, None)
***
production
('ecoinvent 3.4 conseq', '046cbe8c89897c7047b3fad63f0a7279')
'heat and power co-generation unit construction, 500kW electrical, components for electricity only' (unit, RoW, None)
'heat and power co-generation unit construction, 500kW electrical, components for electricity only' (unit, RoW, None)
***
technosphere
('ecoinvent 3.4 conseq', 'f63cad4b06b9edb3311984cca45585a3')
'market for generator, 200kW electrical' (unit, GLO, None)
'heat and power co-generation unit construction, 500kW electrical, components for electricity only' (unit, RoW, None)
***
technosphere
('ecoinvent 3.4 conseq', '8b23dcf0b6f1eaec15cda0a70e71e7d5')
'market for electric parts, h

So the way ecoinvent is written in DB is withouth product manes, only activity names.

In [9]:
for exc in list(myact2.exchanges()):
    print(exc['type'])
    print(exc['input'])
    print(exc.input) # means 'from'
    print(exc.output) # means 'to'
    print('***')

technosphere
('testdb', 'Fuel production')
'Refined fuel' (kg, None, None)
'Electricity, low voltage' (kWh, None, None)
***
biosphere
('testdb', 'Carbon dioxide')
'Carbon dioxide' (kg, None, None)
'Electricity, low voltage' (kWh, None, None)
***
biosphere
('testdb', 'Sulphur dioxide')
'Sulphur dioxide' (kg, None, None)
'Electricity, low voltage' (kWh, None, None)
***
production
('testdb', '12alksdnaslkj')
'Electricity, low voltage' (kWh, None, None)
'Electricity, low voltage' (kWh, None, None)
***


This looks like ecoinvent but I hav ethe names of the products. To be fully consistent I shuold use this format instead

In [14]:
t_db2 = bw.Database("testdb2")
t_db2.write({
    ("testdb2", "12alksdnaslkj"):{   # Here I use a random sequence of nrs and letter as activity code (like in ecoinvent)
        'name':'Electricity production',
        'unit': 'kWh', 
        'exchanges': [{
                'input': ('testdb2', '12239402790sdfa'),
                'amount': 2,
                'unit': 'kg',
                'type': 'technosphere'
            },{
                'input': ('testdb2', 'Carbon dioxide'),
                'amount': 1,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb2', 'Sulphur dioxide'),
                'amount': 0.1,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb2', '12alksdnaslkj'), #important to write the same process name in output
                'amount': 10,
                'unit': 'kWh',
                'type': 'production'
            }]
        },
    ('testdb2', '12239402790sdfa'):{ # Here I use a name as activity name
        'name': 'Fuel production',
        'unit': 'kg',
        'exchanges':[{
                'input': ('testdb2', 'Carbon dioxide'),
                'amount': 10,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb2', 'Sulphur dioxide'),
                'amount': 2,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb2', 'Crude oil'),
                'amount': -50,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('testdb2', '12239402790sdfa'),
                'amount': 1,
                'unit': 'kg',
                'type': 'production'
            }]
    },
    ('testdb2', 'Carbon dioxide'):{'name': 'Carbon dioxide', 'unit':'kg', 'type': 'biosphere'},
    ('testdb2', 'Sulphur dioxide'):{'name': 'Sulphur dioxide', 'unit':'kg', 'type': 'biosphere'},
    ('testdb2', 'Crude oil'):{'name': 'Crude oil', 'unit':'kg', 'type': 'biosphere'}

    })


Writing activities to SQLite3 database:
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/12/2018 09:03:55
  Finished: 03/12/2018 09:03:55
  Total time elapsed: 00:00:00
  CPU %: 109.20
  Memory %: 0.76


In [15]:
myact_v = t_db2.get("12alksdnaslkj")

In [16]:
for exc in list(myact_v.exchanges()):
    print(exc['type'])
    print(exc['input'])
    print(exc.input) # means 'from'
    print(exc.output) # means 'to'
    print('***')

technosphere
('testdb2', '12239402790sdfa')
'Fuel production' (kg, None, None)
'Electricity production' (kWh, None, None)
***
biosphere
('testdb2', 'Carbon dioxide')
'Carbon dioxide' (kg, None, None)
'Electricity production' (kWh, None, None)
***
biosphere
('testdb2', 'Sulphur dioxide')
'Sulphur dioxide' (kg, None, None)
'Electricity production' (kWh, None, None)
***
production
('testdb2', '12alksdnaslkj')
'Electricity production' (kWh, None, None)
'Electricity production' (kWh, None, None)
***
